In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import Spotify
import pandas as pd
import psycopg2
import psycopg2.extras
import os
import sys
import config
import json
import pprint
import glob
from config.conf import conf
from utils.utils import *

### Liked Songs Table

In [34]:
scope_library = 'user-library-read'

sp_auth = SpotifyOAuth(
    client_id = conf['client_id'],
    client_secret = conf['client_secret'],
    redirect_uri = conf['redirect_url'], 
    scope = scope_library
)

sp = Spotify(auth_manager = sp_auth)

# Initialize variables for the While Loop
all_like_songs = []
offset = 0
limit = 50

# Loop over my playlist to retrieve all tracks
while True:
    results = sp.current_user_saved_tracks(limit = limit, offset = offset)
    
    for item in results['items']:
        track = item['track']
        
        track_info = {
            'added_at': item['added_at'],
            'artists': [artist['name'] for artist in track['artists']],
            'name': track['name'],
            'album': track['album']['name'],
            'duration': track['duration_ms']
        }
        all_like_songs.append(track_info)
        
    # Adding results to the list
    all_like_songs.extend(results['items'])
    
    # If fewer than limit 50, then break while loop because loop has reached the end
    if len(results['items']) < limit:
        break
    
    # Increment the offset to get next set of tracks after each loop
    offset += limit

like_songs_df = pd.DataFrame(all_like_songs)
like_songs_clean = like_songs_df.query(' name.notnull() ')
like_songs_clean.drop('track', axis = 1, inplace = True)
like_songs_clean['added_at'] = like_songs_clean['added_at'].str.replace('[a-zA-Z]', ' ', regex = True).str.strip()
like_songs_clean['artists'] = like_songs_clean['artists'].apply(sorted)

# Initializing columns to create table
like_songs_col = like_songs_clean.columns.tolist()
like_songs_types = ['TIMESTAMP', 'TEXT[]', 'VARCHAR(200)', 'VARCHAR(200)', 'NUMERIC']

create_table(like_songs_clean, 'liked_songs', like_songs_col, like_songs_types)

/var/folders/_d/ck_82j6x2vx7b1w6jlm_5cjr0000gn/T/ipykernel_5383/1928061993.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  like_songs_clean.drop('track', axis = 1, inplace = True)
/var/folders/_d/ck_82j6x2vx7b1w6jlm_5cjr0000gn/T/ipykernel_5383/1928061993.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  like_songs_clean['added_at'] = like_songs_clean['added_at'].str.replace('[a-zA-Z]', ' ', regex = True).str.strip()
/var/folders/_d/ck_82j6x2vx7b1w6jlm_5cjr0000gn/T/ipykernel_5383/1928061993.py:47: SettingWithCopyWarning: 
A value is trying to be set on a c

### Streaming History Table

In [3]:
json_list = []

for jsons in glob.glob('Spotify Account Data/StreamingHistory_music*.json'):
    with open(jsons, 'r') as file:
        data  = json.load(file)
        
    df = pd.DataFrame(data)
    
    # Creating TIMESTAMP columns
    df['endTime_full'] = pd.to_datetime(df['endTime'], format = '%Y-%m-%d %H:%M')
    df['startTime'] = df['endTime_full'] - pd.to_timedelta(df['msPlayed'], unit = 'ms')
    
    # Creating ArtistName ARRAY column
    df['artistNameArray'] = df['trackName'].str.extract(r'\(feat\.\s*(.*?)\)')
    df['artistName'] = df[['artistName', 'artistNameArray']].values.tolist()
    
    def remove_nan(col):
        return [x for x in col if not pd.isna(x)]
    
    df['artistName'] = df['artistName'].apply(remove_nan)
    df['artistName'] = df['artistName'].apply(sorted)
    
    df = df[['trackName', 'artistName', 'endTime', 'startTime', 'msPlayed']]
    
    json_list.append(df)

streaming_hist_df = pd.concat(json_list)

streaming_hist_col = streaming_hist_df.columns.tolist()
streaming_hist_types = ['VARCHAR(200)', 'TEXT[]', 'TIMESTAMP', 'TIMESTAMP', 'NUMERIC']

create_table(streaming_hist_df, 'streaming_hist', streaming_hist_col, streaming_hist_types)

### Search Query Table

In [24]:
with open('Spotify Account Data/SearchQueries.json', 'r') as file:
        data  = json.load(file)

search_df = pd.DataFrame(data)[['searchTime', 'searchQuery']]
search_df['searchTime'] = search_df['searchTime'].str.rstrip(to_strip = 'Z[UTC]').str.replace('T', ' ')


search_df_col = search_df.columns.tolist()
search_df_types = ['TIMESTAMP', 'VARCHAR(200)']

create_table(search_df, 'search_query', search_df_col, search_df_types)

In [ ]:
# def create_table(table_name):
#     try:
#         conn = psycopg2.connect(host = conf.host, dbname = conf.dbname, user = conf.user, password = conf.password, port = conf.port)
#         cur = conn.cursor()

#         cur.execute(""" DROP TABLE IF EXISTS liked_songs """)
        
#         cur.execute("""
#                     CREATE TABLE IF NOT EXISTS liked_songs(
#                                             added_date VARCHAR(200),
#                                             artists TEXT[],
#                                             song_name VARCHAR(200)
#                     )
#                     """)
        
#         insert_query = """
#             INSERT INTO liked_songs (added_date, artists, song_name)
#             VALUES (%s, %s, %s)
#         """
        
#         for _, row in df_clean.iterrows():
#             cur.execute(insert_query, (row['added_at'], row['artists'], row['name']))


#         conn.commit()

#         cur.close()
#         conn.close()
#     except Exception as error:
#         print(error)

In [ ]:
# def create_table(df, table_name, column_list, column_schema_list):
#     """
    
#     Creating a PostgreSQL table from Pandas DataFrame
#     Parameters:
#         DF: Pandas DataFrame (clean Dataframe beforehand so the input is the intended dtype)
#         Table_name: Name shown in Postgres
#         Column_list: List of column names in DataFrame
#         Column_schema_list: List of dtypes from DataFrame that must match the order of column_list
        
#     """
    
#     try:
#         # Connect to the database
#         conn = psycopg2.connect(
#             host=conf.host, 
#             dbname=conf.dbname, 
#             user=conf.user, 
#             password=conf.password, 
#             port=conf.port
#         )
#         cur = conn.cursor()

#         # Drop table if it exists
#         cur.execute(f""" 
#                     DROP TABLE IF EXISTS {table_name}
#                     """)

#         # Construct CREATE TABLE dynamically
#         columns_with_types = ', '.join([f'{col} {dtype}' for col, dtype in zip(column_list, column_schema_list)])
#         create_table_query = f"""
#                     CREATE TABLE IF NOT EXISTS {table_name} ({columns_with_types})
#                     """

#         # Execute CREATE TABLE 
#         cur.execute(create_table_query)

#         # Construct INSERT query dynamically
#         placeholders = ", ".join(["%s"] * len(column_list))
#         insert_query = f"""
#                     INSERT INTO {table_name} ({', '.join(column_list)}) VALUES ({placeholders})
#                      """

#         # Insert rows from dataframe
#         for _, row in df.iterrows():
#             cur.execute(insert_query, tuple(row[col] for col in column_list))


#         conn.commit()
#         cur.close()
#         conn.close()
#     except Exception as error:
#         print(error)


# columns = ['added_at', 'artists', 'name']
# column_types = ['VARCHAR(200)', 'TEXT[]', 'VARCHAR(200)']

# create_table(df_clean, 'liked_songs', columns, column_types)